In [1]:
import sys
sys.version

'3.5.2 |Anaconda 4.1.1 (x86_64)| (default, Jul  2 2016, 17:52:12) \n[GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]'

In [2]:
import justext
import requests
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.probability import FreqDist
from heapq import nlargest
from bs4 import BeautifulSoup

In [22]:
url = 'http://www.space.com/35024-nasa-spacex-red-dragon-mars-mission.html'
response = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})

In [23]:
response.content

b'<!DOCTYPE html>\n<html>\n<head>\n    <!-- META TAGS -->\n    <meta charset="UTF-8">\n    \n            \n<meta name="description" content="NASA doesn&amp;#039;t plan to put any science instruments aboard SpaceX&amp;#039;s first Mars mission, which could launch as early as 2018.">\n\n    <title>SpaceX&#039;s 1st Mars Mission Won&#039;t Carry NASA Science Gear</title>\n\n<meta http-equiv="X-UA-Compatible" content="IE=9; IE=8; IE=7; IE=EDGE">\n<meta name="google-site-verification" content="MbXIxQZKTmppqEDFE0bdWHdQq_OSDiASMxZeFX1x-3k">    \n        \n            <meta name="viewport" content="width=device-width, initial-scale=1">\n    \n    <!--[if IE ]>\n\n    <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>\n\n    <![endif]-->\n\n            <meta name="robots" content=" index, follow ">\n    \n            <link rel="canonical" href="http://www.space.com/35024-nasa-spacex-red-dragon-mars-mission.html">\n\n    <link rel="amphtml" href="http://amp.space.com/35024-nas

In [24]:
soup = BeautifulSoup(response.content, 'html.parser')

In [25]:
metas = soup.find_all('meta')
metas

[<meta charset="utf-8">
 <meta content="NASA doesn&amp;#039;t plan to put any science instruments aboard SpaceX&amp;#039;s first Mars mission, which could launch as early as 2018." name="description">
 <title>SpaceX's 1st Mars Mission Won't Carry NASA Science Gear</title>
 <meta content="IE=9; IE=8; IE=7; IE=EDGE" http-equiv="X-UA-Compatible">
 <meta content="MbXIxQZKTmppqEDFE0bdWHdQq_OSDiASMxZeFX1x-3k" name="google-site-verification">
 <meta content="width=device-width, initial-scale=1" name="viewport">
 <!--[if IE ]>
 
     <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
 
     <![endif]-->
 <meta content=" index, follow " name="robots">
 <link href="http://www.space.com/35024-nasa-spacex-red-dragon-mars-mission.html" rel="canonical">
 <link href="http://amp.space.com/35024-nasa-spacex-red-dragon-mars-mission.html" rel="amphtml">
 <link href="/resources/apple-icon-57x57.png" rel="apple-touch-icon" sizes="57x57">
 <link href="/resources/apple-icon-60x60.png" rel=

In [26]:
meta_summary = ''
for meta in metas:
    try:
        if meta['property'] == 'og:description':
            meta_summary = meta['content']
    except:
        pass
print(meta_summary)

NASA doesn't plan to put any science instruments aboard SpaceX's first Mars mission, which could launch as early as 2018.


In [27]:
sent_tokenize(meta_summary)

["NASA doesn't plan to put any science instruments aboard SpaceX's first Mars mission, which could launch as early as 2018."]

In [28]:
sent_meta_summary = sent_tokenize(meta_summary)
if len(sent_meta_summary) > 1:
    del sent_meta_summary[-1]
meta_summary = '\n'.join(sent_meta_summary)
print(meta_summary)

NASA doesn't plan to put any science instruments aboard SpaceX's first Mars mission, which could launch as early as 2018.


In [29]:
paragraphs = justext.justext(response.content, justext.get_stoplist("English"))
text = ""
for paragraph in paragraphs:
    if not paragraph.is_boilerplate and not paragraph.is_heading:
        text += paragraph.text

In [30]:
text

'SAN FRANCISCO — NASA doesn\'t plan to put any science instruments aboard SpaceX\'s first Mars mission, which could launch as early as 2018, agency officials said.NASA wants to wait until SpaceX proves it can pull off a soft landing on the Red Planet before committing millions of dollars\' worth of equipment to the spaceflight company\'s "Red Dragon" effort, said Jim Green, head of the agency\'s Planetary Science Division."Landing on Mars is hard," Green said during a talk Tuesday (Dec. 13) here at the annual fall meeting of the American Geophysical Union (AGU). "I want to wait this one out." [SpaceX\'s Red Dragon Mars Mission in Images]SpaceX founder and CEO Elon Musk has said the company plans to launch uncrewed Dragon capsules toward the Red Planet at every launch opportunity for the foreseeable future, ideally beginning in 2018. (Such windows come just once every 26 months, when Earth and Mars align favorably.)The main goal is to practice landing heavy payloads on Mars, to help pav

In [31]:
sents = sent_tokenize(text)

In [32]:
word_sent = word_tokenize(text.lower())

In [33]:
_stopwords = set(stopwords.words('english') + list(punctuation))

In [34]:
word_sent=[word for word in word_sent if word not in _stopwords]

In [35]:
freq = FreqDist(word_sent)

In [36]:
nlargest(10, freq, key=freq.get)

['mars',
 'red',
 "'s",
 'dragon',
 'agency',
 'landing',
 "''",
 'said',
 '``',
 'spacex']

In [37]:
from collections import defaultdict
ranking = defaultdict(int)

for i,sent in enumerate(sents):
    for w in word_tokenize(sent.lower()):
        if w in freq:
            ranking[i] += freq[w]

In [38]:
sents_idx = nlargest(4, ranking, key=ranking.get)

In [39]:
summary = meta_summary
summary_sents = [sents[j] for j in sorted(sents_idx)]
if len(summary_sents) > 0:
    summary = [summary] + summary_sents

In [40]:
print('\n'.join(summary))

NASA doesn't plan to put any science instruments aboard SpaceX's first Mars mission, which could launch as early as 2018.
SAN FRANCISCO — NASA doesn't plan to put any science instruments aboard SpaceX's first Mars mission, which could launch as early as 2018, agency officials said.NASA wants to wait until SpaceX proves it can pull off a soft landing on the Red Planet before committing millions of dollars' worth of equipment to the spaceflight company's "Red Dragon" effort, said Jim Green, head of the agency's Planetary Science Division.
[SpaceX's Red Dragon Mars Mission in Images]SpaceX founder and CEO Elon Musk has said the company plans to launch uncrewed Dragon capsules toward the Red Planet at every launch opportunity for the foreseeable future, ideally beginning in 2018.
The Red Dragon missions also should have significant scientific returns, since they'll allow researchers to send a variety of gear to the Red Planet, Musk has said.NASA will take advantage of such opportunities, a